In [1]:
from search_methods import *
from openai import AzureOpenAI 

/Users/muhammadarham/Drive/TraversaalCode/QAMCode/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----- Complete loading inputs -----
['hornby', 'funkybuys', 'ccf', 'generic', 'chuggington', 'kato (usa)', 'bachmann', 'kato', 'power trains', 'preiser', 'takara tomy', 'roco', 'fisher-price', 'gaugemaster', 'faller', 'topro', 'dapol', 'ratio', 'arsuk', 'trix', 'railway - bachmann', 'busch', 'piko', 'corgi', 'sohni-wicke', 'train tech', 'cato', 'power city trains', 'train', 'sales la', 'märklin my world', 'mattel', 'graham farish', 'carousel', 'peptim scenics', 'learning curve', 'walthers trainline', 'woodland scenics', 'peco', 'fisher price', 'jouef', 'thomas & friends', 'plan toys', 'bandai', 'atlas', 'nmit', 'viessmann', 'tommy tech', 'model power', 'micki leksaker', 'lelin', 'bachmann n gauge', 'märklin', 'tomytec', 'bigjigs rail', 'trains', 'tomy', 'smasha-ballz', 'moomins', 'paddington bear', 'charlie bears', 'yoohoo & friends', 'monchhichi', 'aurora', 'the puppet company', 'ty', 'henry hugglemonster', 'melissa & doug', 'animal babies', 'olympic mascots', 'suma collection', 'whit

/Users/muhammadarham/Drive/TraversaalCode/QAMCode/Evaluation/search_methods.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.price_in_usd.fillna(df.price_in_usd.mean(), inplace=True)
/Users/muhammadarham/Drive/TraversaalCode/QAMCode/Evaluation/search_methods.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [2]:
df, corpus, corpus_embeddings_dict, corpus_processed = load_inputs(
    df_filepath='../Data/Toydata_w_new_att_final.xlsx', embedding_filepath='../Model/corpus_embeddings.pkl', processed_corpus_filepath='../Model/corpus_processed.pkl')

embedder, cross_encoder = load_models()

llm_client = AzureOpenAI(
    api_key="",
    api_version="2024-02-15-preview",
    azure_endpoint="",
    azure_deployment="gpt-4o-large"
)


/Users/muhammadarham/Drive/TraversaalCode/QAMCode/Evaluation/search_methods.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.price_in_usd.fillna(df.price_in_usd.mean(), inplace=True)
/Users/muhammadarham/Drive/TraversaalCode/QAMCode/Evaluation/search_methods.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [3]:
print(embedder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: NomicBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [4]:
import json

with open('matched_queries.json', 'r') as f:
    queries = json.load(f)

queries = [query.lower() for query in queries]

In [5]:
queries

["what's a good gift for a 16-year-old that's highly rated?",
 "looking for a 5-star rated item suitable for someone who's almost 16.",
 'looking for a cat construction toy priced at $12 with a rating of 4.7.',
 'can i find papo products with a rating of 4.7?',
 'looking for a highly-rated product by spektrum, any suggestions?',
 'looking for highly rated products by stevenson entertainment, any suggestions?',
 'where can i find toys4u products with a 5-star rating?',
 'looking for a kaleidoscope toy for my 3-year-old, priced around $12.',
 "how well-rated is the accessotech product that's been on the market for a couple of years?",
 'are there any reviews for the classified product priced at $16.0?',
 'are there any lifewheel items that have a 4.5-star rating?',
 'are there any reviews for k2-accessories alphabet letter beads with a 4.2 rating?',
 'looking for a rolly toys product with a rating of 4.0.',
 'are there any highly-rated advent gift boxes available for teenagers?',
 'is th

In [6]:
def get_results(hits, top_n=5):
    results = []
    result_dict = {}
    for idx in hits.keys():  # loop through hits
        product = df['product_name'].loc[(df['review_id'] == idx)].tolist()[0]
        review = df['reviews'].loc[(df['review_id'] == idx)].tolist()[0]
        if product not in result_dict:
            if len(results) < top_n:
                result_dict[product] = len(results)
                product_df = df.loc[(df['product_name'] == product)]

                results.append({
                    'product_name': product,
                    'manufacturer': product_df['manufacturer'].unique()[0],
                    'price': round(product_df['price_in_usd'].mean(), 2),
                    'preferred_age': round(product_df['preferred_age'].min(), 0),
                    'rating': round(product_df['star_rating'].max(), 1),
                    'reviews': [review]
                })
        else:
            results[result_dict[product]]['reviews'].append(review)

    return results

In [7]:
import random

query = random.choice(queries)
print(query)

can i find products around $3.00 on this site?


## Bi-Encoder Search

In [8]:
biencoder_hits = search_reviews(query, corpus_embeddings_dict, embedder, 10)
res = get_results(biencoder_hits, top_n=30)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


In [9]:
print(type(res))
print(len(res))


<class 'list'>
10


In [10]:
for hit in res:
    print(hit)
    # print(res[hit])
    print("-"*100)

{'product_name': 'Le Toy Van London Car Set', 'manufacturer': 'le toy van', 'price': np.float64(18.44), 'preferred_age': np.float64(3.0), 'rating': np.float64(4.3), 'reviews': ['Good toy for a 3 year old, but price found cheaper elsewhere (in a toystore) afterwards.  Lovely set of cars']}
----------------------------------------------------------------------------------------------------
{'product_name': 'PLAYMOBIL 7451 - 3 ouvriers de chantiers avec accessoires', 'manufacturer': 'playmobil', 'price': np.float64(12.29), 'preferred_age': np.float64(3.0), 'rating': np.float64(5.0), 'reviews': ['£16 ????? what a rip off.I came on Amazon to see if i could get this item for a good price.You can buy this for £8 on the Playmobil site[...]']}
----------------------------------------------------------------------------------------------------
{'product_name': 'Cars Character Car - Dirt Track McQueen', 'manufacturer': 'disney', 'price': np.float64(13.52), 'preferred_age': np.float64(3.0), 'ratin

## Cross Encoder Search

In [11]:
cross_encoder_hits = cross_encoder_search(query, corpus_embeddings_dict, embedder, cross_encoder, top_k=10, top_n=10)
res = get_results(cross_encoder_hits, top_n=10)

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.11it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
print(cross_encoder_hits)

{32655: np.float32(0.0987966), 16002: np.float32(0.0032346717), 31930: np.float32(0.0011469306), 4547: np.float32(0.0003140491), 34362: np.float32(0.00029673183), 28902: np.float32(0.00019542727), 16406: np.float32(0.00015687014), 7320: np.float32(0.00014724505), 32802: np.float32(0.0001416004), 26483: np.float32(0.00010534998)}


In [13]:
for hit in res:
    print(hit)
    print("-"*100)

{'product_name': 'PLAYMOBIL 7451 - 3 ouvriers de chantiers avec accessoires', 'manufacturer': 'playmobil', 'price': np.float64(12.29), 'preferred_age': np.float64(3.0), 'rating': np.float64(5.0), 'reviews': ['£16 ????? what a rip off.I came on Amazon to see if i could get this item for a good price.You can buy this for £8 on the Playmobil site[...]']}
----------------------------------------------------------------------------------------------------
{'product_name': 'Cars Character Car - Dirt Track McQueen', 'manufacturer': 'disney', 'price': np.float64(13.52), 'preferred_age': np.float64(3.0), 'rating': np.float64(4.3), 'reviews': ['MY SON IS A HUGE FAN OF THE PIXAR CHARACTER CARS I FOUND THIS SITE THE BEST FOR CHOICE, VALUE ,AND AVAILABILITY. THEY OFFER MORE THAN IS AVAILABLE IN THE SHOPS AS THEY HAVE ALOT MORE CHARACTERS ON OFFER AND BELIEVE ME, I HAVE SEARCHED FOR THEM.  Be careful what you wish for']}
-------------------------------------------------------------------------------

## BM25 Keyword Search

In [14]:
keyword_results = bm25_search_reviews(query, corpus_processed, top_n=10)
res = get_results(keyword_results, top_n=10)

In [15]:
for hit in res:
    print(hit)
    # print(res[hit])
    print("-"*100)

{'product_name': 'WREATH HOOK SUCTION CUP', 'manufacturer': 'christmas and decorations', 'price': np.float64(2.45), 'preferred_age': np.float64(3.0), 'rating': np.float64(4.1), 'reviews': ['Just use a little gel around the edges and you will find the hook stays up fine. The gel I have used is not watwerproof ...']}
----------------------------------------------------------------------------------------------------
{'product_name': 'LEGO Star Wars: Biker Scout Trooper Minifigure with Blaster Rifle', 'manufacturer': 'lego', 'price': np.float64(9.22), 'preferred_age': np.float64(3.0), 'rating': np.float64(5.0), 'reviews': ['Arrived quickly and well packaged in time for christmas. Searched around for sometime to find this particular piece. Will happily use again.  5 out of 5']}
----------------------------------------------------------------------------------------------------
{'product_name': 'BrainBox - Art', 'manufacturer': 'the green board game co.', 'price': np.float64(7.37), 'preferr

## Hybrid Search

In [16]:
response, matched = hybrid_search(query, df, corpus_embeddings_dict, llm_client=llm_client, embedder=embedder, cross_encoder=cross_encoder, top_k=25, top_n=10)
res = get_results(response, top_n=10)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


In [17]:
for hit in res:
    print(hit)
    print("-"*100)

{'product_name': "Holographic Self-Adhesive Dots Embellishments for Children's Crafts, Collage, Scrapbooking (Per Pack)", 'manufacturer': 'baker ross', 'price': np.float64(3.68), 'preferred_age': np.float64(3.0), 'rating': np.float64(4.0), 'reviews': ['Good craft item for children to have around, no need for glue.']}
----------------------------------------------------------------------------------------------------
{'product_name': 'Disney Princess Birthday Candle Number 3', 'manufacturer': 'disney princess', 'price': np.float64(3.69), 'preferred_age': np.float64(3.0), 'rating': np.float64(4.5), 'reviews': ['The face looked a bit strange but with industrial promotion what can you do. Next time I would go to a shop and choose it myself. In any case makes the job.  Snow White 3 candle']}
----------------------------------------------------------------------------------------------------
{'product_name': 'Spaceboy Water Pistol', 'manufacturer': 'dotcomgiftshop', 'price': np.float64(2.4),

## Search Results

In [18]:
import json

with open('matched_queries.json', 'r') as f:
    queries = json.load(f)

queries = [query.lower() for query in queries]
random.shuffle(queries)

In [19]:
from tqdm import tqdm
res = []
for query in tqdm(queries):
    print(query)
    biencoder_hits = search_reviews(query, corpus_embeddings_dict, embedder, 10)
    biencoder_results = get_results(biencoder_hits, top_n=10)

    cross_encoder_hits = cross_encoder_search(query, corpus_embeddings_dict, embedder, cross_encoder, top_k=10, top_n=10)
    cross_encoder_results = get_results(cross_encoder_hits, top_n=10)

    keyword_results = bm25_search_reviews(query, corpus_processed, top_n=10)
    keyword_results = get_results(keyword_results, top_n=10)
    
    response, matched = hybrid_search(query, df, corpus_embeddings_dict, llm_client=llm_client, embedder=embedder, cross_encoder=cross_encoder, top_k=10, top_n=10);
    hybrid_search_results = get_results(response, top_n=10)
    # print(response)
    if len(hybrid_search_results) != 0:
        res.append({
            "query": query,
            "cross_encoder_search_results": cross_encoder_results,
            "biencoder_search_results": biencoder_results,
            "keyword_search_results": keyword_results,
            "hybrid_search_results": hybrid_search_results,
        })
    print("-"*100)

  0%|          | 0/233 [00:00<?, ?it/s]

can i get a 5-star rated ban dai toy for my almost 4-year-old?


  0%|          | 1/233 [00:04<15:44,  4.07s/it]

----------------------------------------------------------------------------------------------------
what are some good reviews on the glowhouse products priced at $3?


Batches: 100%|██████████| 1/1 [00:00<00:00, 29.90it/s]


...

----------------------------------------------------------------------------------------------------
what's the rating for products from ultrabyeasypeasystore?


100%|█████████▉| 232/233 [19:08<00:05,  5.97s/it]

----------------------------------------------------------------------------------------------------
looking for a 5-star rated item from footprintdirect.co.uk that's affordable?


100%|██████████| 233/233 [19:13<00:00,  4.95s/it]

----------------------------------------------------------------------------------------------------


In [20]:
import json
with open('eval.json', 'w') as f:
    json.dump(res, f, indent=4)